In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import pprint

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from iminuit import Minuit
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

# initialize a nparray: best way is to know the size in advance
# MFPAD_RCR=np.zeros((72,2))
# ctetha_RCR=np.zeros((72,2))

MFPAD_RCR=[] # initilialises a list
ctetha_RCR=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCR[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm))
            cosphi.append((ctheta_nm+0.1666,phi_nm+15)) # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCR[filename].numpy())
            MFPAD_RCR.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCR[filename].numpy())
            ctetha_RCR.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCR=np.array(MFPAD_RCR)
ctetha_RCR=np.array(ctetha_RCR) # it is a list!

MFPAD_RCL=[] # initilialises a list
ctetha_RCL=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCL[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm)) #they are inverted
            cosphi.append((ctheta_nm+0.1666,phi_nm+15))  # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCL[filename].numpy())
            MFPAD_RCL.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCL[filename].numpy())
            ctetha_RCL.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCL=np.array(MFPAD_RCL)
ctetha_RCL=np.array(ctetha_RCL) # it is a list!

ctheta_cred = (ctheta_c[1:] + ctheta_c[:-1])/2

In [ ]:
print(len(MFPAD_RCL[0].T))
print(MFPAD_RCL[0].T[0])
print(len(MFPAD_RCL[0].T[0]))

x_bins = np.linspace(-1,1, len(MFPAD_RCL[0].T[0]))

plt.scatter(x_bins,MFPAD_RCL[0].T[0],color='r')
plt.errorbar(x_bins,MFPAD_RCL[0].T[0], yerr=np.sqrt(MFPAD_RCL[0].T[0]), fmt="o")
# plt.plot()

In [ ]:
class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        z = (self.y - ym)/(self.err**2)
        return np.sum(z**2)

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

#INTESITY NOT PECD
def INTENSITY2(x, b1, b2):
    return 2 + 2*b1*x - 0.5*b2*(3*x**2-1)

def INTENSITY2_cross(x, A, b1, b2):
    return A/(4*np.pi)*(1 + b1*x - 0.25*b2*(3*x**2-1))


# param_matrix=np.zeros((6,72)) #nparray initialization
param_matrix = [[0 for i in range(6)] for j in range(3)] #i col, j row

for col, i in zip(MFPAD_RCR[0].T,range(len(MFPAD_RCR[0].T))):
    x_data = np.linspace(-1, 1, len(col))
    y_data = col
    y_err = np.sqrt(col)
    lsq = BetterLeastSquares(INTENSITY2_cross, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1))
    m.migrad()
    m.hesse() # run covariance estimator
    # print(m.params)
    # for p in m.parameters:
        # print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))
    for j,p in zip(range(len(m.parameters)),m.parameters):
        param_matrix[j][i] = ((m.values[p], m.errors[p]))

In [ ]:
x_data = np.linspace(-1, 1, len(MFPAD_RCL[0].T[0]))
y_data = MFPAD_RCL[0].T[0]
y_err = np.sqrt(MFPAD_RCL[0].T[0])
lsq = BetterLeastSquares(INTENSITY2_cross, x_data, y_data, y_err)

m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1))
m.migrad()

In [ ]:
param_matrix

In [ ]:
b1=[col[0] for col in param_matrix[1]]
b1_err=[col[1] for col in param_matrix[1]]

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
x_data_new = np.linspace(-1, 1, 100)
bspl = make_interp_spline(x_data, INTENSITY2_cross(x_data, *m.values.values()), k=3)
plt.plot(x_data_new, bspl(x_data_new))
plt.scatter(x_bins,MFPAD_RCL[0].T[0],color='r')
plt.errorbar(x_bins,MFPAD_RCL[0].T[0], yerr=np.sqrt(MFPAD_RCL[0].T[0]), fmt="o")


In [ ]:
#b1 trends across the phi in MFPAD
x_data_b1=np.linspace(-1,1,len(b1))
plt.scatter(x_data_b1,b1,color='r')
plt.errorbar(x_data_b1,b1, yerr=b1_err, fmt="o")
plt.show()

In [ ]:
class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        z = (self.y - ym)/(self.err**2)
        return np.sum(z**2)

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

#INTESITY NOT PECD
def INTENSITY2(x, b1, b2):
    return 2 + 2*b1*x - 0.5*b2*(3*x**2-1)

def INTENSITY2_cross(x, A, b1, b2):
    return A/(4*np.pi)*(1 + b1*x - 0.25*b2*(3*x**2-1))

# param_matrix=np.zeros((6,72)) #nparray initialization
param_matrix = [[[0 for j in range(3)] for i in range(6)] for z in range(72)] #i col, j row, z height

# pprint.pprint(param_matrix)

for row, z in zip(MFPAD_RCR,range(len(MFPAD_RCR))): # len = 72
    for col, i in zip(MFPAD_RCR[z].T,range(len(MFPAD_RCR[z].T))): # len = 6
        x_data = np.linspace(-1, 1, len(col))
        y_data = col
        y_err = np.sqrt(col)
        lsq = BetterLeastSquares(INTENSITY2_cross, x_data, y_data, y_err)

        m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1))
        m.migrad()
        m.hesse() # run covariance estimator
        # print(m.params)
        # for p in m.parameters:
            # print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))
        for j,p in zip(range(len(m.parameters)),m.parameters): # len = 3
            param_matrix[z][i][j] = ((m.values[p], m.errors[p]))

In [ ]:
 b1=[col[1][1] for col in param_matrix[8]]
 b1

In [ ]:
from scipy.interpolate import interp1d

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
custom_ylim = (-0.9,0.9)
x_data_new_b1 = np.linspace(-1, 1, 50)

for i,axes in zip(range(72),axes.flatten()):
    b1=[col[1][0] for col in param_matrix[i]]
    b1_err=[col[1][1] for col in param_matrix[i]]
    # spl = BSpline(x_data_b1, b1, k=4)
    f2=interp1d(x_data_b1,b1, kind='cubic')
    axes.plot(x_data_new_b1, f2(x_data_new_b1), 'b-', lw=4, alpha=0.4, label='cubic')
    axes.scatter(x_data_b1,b1,color='r')
    axes.set_xlim(custom_xlim)
    axes.set_ylim(custom_ylim)
    axes.errorbar(x_data_b1,b1, yerr=b1_err, fmt="o")
fig.tight_layout()